<a href="https://colab.research.google.com/github/nmaffly/rhythmoji/blob/main/rhythmoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Load the dataset
file_path = '/content/enhanced_fashion_dataset (1).csv'  # Replace with your file path
data = pd.read_csv(file_path)

# One-hot encode the genre columns
genre_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
genre_features = genre_encoder.fit_transform(data[['Genre_1', 'Genre_2', 'Genre_3']])

# Normalize your input features
scaler = StandardScaler()
genre_features_scaled = scaler.fit_transform(genre_features)

# Determine the number of features in X_train
num_features = genre_features_scaled.shape[1]

target_columns = ['Shirt', 'Shoes', 'Sweater', 'Pants', 'Accessories']

# Global factorization and one-hot encoding
factorized_targets = {col: pd.factorize(data[col])[0] for col in target_columns}
num_categories = {col: len(set(factorized_targets[col])) for col in target_columns}
Y = {col: to_categorical(factorized_targets[col], num_classes=num_categories[col]) for col in target_columns}

# Reformat target data
Y_combined = list(zip(*[Y[col] for col in target_columns]))

# Split the dataset
X_train, X_test, Y_train_combined, Y_test_combined = train_test_split(genre_features_scaled, Y_combined, test_size=0.2, random_state=42)

# Convert the tuples back into separate arrays for training the model
Y_train = {col: np.array([y[i] for y in Y_train_combined]) for i, col in enumerate(target_columns)}
Y_test = {col: np.array([y[i] for y in Y_test_combined]) for i, col in enumerate(target_columns)}


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
num_categories[col]

367

In [5]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

# Assuming num_features is already defined from your previous code
input_layer = Input(shape=(num_features,))

# Neural network layers
x = Dense(256, activation='relu')(input_layer)  # More neurons
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.3)(x)  # Adjust dropout rate
x = Dense(128, activation='relu')(x)  # Additional layer
x = Dropout(0.3)(x)
output_layers = [Dense(num_categories[col], activation='softmax')(x) for col in target_columns]

model = Model(inputs=input_layer, outputs=output_layers)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fit the model
history = model.fit(
    X_train, list(Y_train.values()),
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/100
8/8 [==============================] - 0s 28ms/step - loss: 27.5646 - dense_3_loss: 5.4737 - dense_4_loss: 5.5191 - dense_5_loss: 5.5342 - dense_6_loss: 5.5393 - dense_7_loss: 5.4982 - dense_3_accuracy: 0.0312 - dense_4_accuracy: 0.0234 - dense_5_accuracy: 0.0234 - dense_6_accuracy: 0.0195 - dense_7_accuracy: 0.0195 - val_loss: 30.4406 - val_dense_3_loss: 6.0405 - val_dense_4_loss: 6.0702 - val_dense_5_loss: 6.0842 - val_dense_6_loss: 6.1226 - val_dense_7_loss: 6.1231 - val_dense_3_accuracy: 0.0000e+00 - val_dense_4_accuracy: 0.0000e+00 - val_dense_5_accuracy: 0.0000e+00 - val_dense_6_accuracy: 0.0000e+00 - val_dense_7_accuracy: 0.0000e+00
Epoch 2/100
8/8 [==============================] - 0s 17ms/step - loss: 25.6541 - dense_3_loss: 5.1811 - dense_4_loss: 5.1093 - dense_5_loss: 5.0943 - dense_6_loss: 5.1168 - dense_7_loss: 5.1526 - dense_3_accuracy: 0.0430 - dense_4_accuracy: 0.0508 - dense_5_accuracy: 0.0625 - dense_6_accuracy: 0.0664 - dense_7_accuracy: 0.0508 - val_loss

In [13]:
# Evaluate the model
evaluations = model.evaluate(X_test, list(Y_test.values()))

# Correctly print out the loss and accuracy for each target category
for i, col in enumerate(target_columns):
    print(f"Evaluation for {col}: Loss = {evaluations[i * 2]}, Accuracy = {evaluations[i * 2 + 1]}")


3/3 [==============================] - 0s 13ms/step - loss: 30.4366 - dense_3_loss: 6.1414 - dense_4_loss: 6.0619 - dense_5_loss: 6.1184 - dense_6_loss: 6.0211 - dense_7_loss: 6.0939 - dense_3_accuracy: 0.0000e+00 - dense_4_accuracy: 0.0000e+00 - dense_5_accuracy: 0.0000e+00 - dense_6_accuracy: 0.0000e+00 - dense_7_accuracy: 0.0000e+00
Evaluation for Shirt: Loss = 30.436609268188477, Accuracy = 6.141395568847656
Evaluation for Shoes: Loss = 6.06186056137085, Accuracy = 6.118361473083496
Evaluation for Sweater: Loss = 6.021080017089844, Accuracy = 6.093911647796631
Evaluation for Pants: Loss = 0.0, Accuracy = 0.0
Evaluation for Accessories: Loss = 0.0, Accuracy = 0.0


In [21]:
import numpy as np

# Example input genres
input_genres = [['Classical', 'Pop', 'Experimental']]  # Replace with your genres


# Encoding the input genres
input_encoded = genre_encoder.transform(input_genres)

# Make predictions
predictions = model.predict(input_encoded)

# Decoding the predictions
predicted_categories = {col: np.argmax(pred, axis=1)[0] for col, pred in zip(target_columns, predictions)}

# Mapping numerical predictions back to category names
# This step assumes that the categories in the training data are not numerically encoded
# If they are, you will need to use the appropriate mapping
decoded_predictions = {col: data[col].unique()[predicted_categories[col]] for col in target_columns}

# Print out the predicted categories
for category, prediction in decoded_predictions.items():
    print(f"Predicted {category}: {prediction}")


1/1 [==============================] - 0s 31ms/step
Predicted Shirt: Unique Purple Patterned Shirt
Predicted Shoes: Novel Yellow Ankle Boots
Predicted Sweater: Relaxed Blue Hoodie
Predicted Pants: Earthy Orange Blue Jeans
Predicted Accessories: Eclectic Green Baseball Cap


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
